In [ ]:
import sys
import importlib

#sys.path.insert(0, '')
#from PreProcesamiento import *
#importlib.reload

import time
start = time.time()

In [ ]:
#funcion que carga un fichero en un dataframe con el id del usuario en una columna y sus eventos en formato json en otra
def readDataFile(filename):
    import json
    import pandas as pd
    # Leemos el fichero por lineas
    with open(filename) as f:
        users_events = f.readlines()
        
    # Parseamos a formato json cada linea
    listJson = list(map(json.loads, users_events));
   
    # Creamos el dataframe a partir de               
    df = pd.DataFrame.from_dict(listJson, orient='columns');
                               
    return df

In [ ]:
#Funcion que filtra los usuarios en funcion de un numero de eventos pasado por argumento
def dataFiltering(eventosDF, numEventos):           
    return eventosDF[eventosDF['Eventos'].apply(lambda x: len(x) > numEventos)]


In [ ]:
#Funcion que filtra los eventos es funcion de la lista eventos pasados por argumento
# eventsFiltering son por los eventos que queremos filtrar
# listEvents es la lista de eventos del usuario
def filterListByEvents(listEvents, eventsFiltering):
    import pandas as pd
    import numpy
    
    # Comprobacion de si la lista esta vacia (si lo esta, daria error la funcion)
    if listEvents == []:
        return []
    #Pasa la lista de eventos a un dataframe
    df = pd.DataFrame(listEvents)
     
    #comprueba si los eventos del estudiantes estan en la lista de los eventos por los que se filtra    
    df = df.loc[df['evento'].isin(eventsFiltering)]
    
    indices = numpy.array(df.index.values)
    
    listaF = numpy.array(listEvents)
    
    return list(listaF[indices])
    
def eventsFiltering(eventsDF, listEvents):
    # Filtra un dataframe de eventos generando un nuevo dataframe con los eventos
    # pasados por el argumento events
    
    eventsDFFiltered = eventsDF.copy()
    eventsDFFiltered['Eventos'] = eventsDF['Eventos'].apply(lambda x: filterListByEvents(x, listEvents))
    
    return eventsDFFiltered

#Guarda dfToSave en el fichero filename
def dfToCsv(dfToSave,filename):

    dfToSave.to_csv(filename, sep='|', encoding='utf-8', index=False)

#Carga el fichero filename a un dataframe    
def csvToDf(filename):

    csvDf = pd.read_csv(filename,sep='|')
    
    return csvDf


In [ ]:
"""
Descartado por consumo de tiempo

def filterListByDate(listEvents, dates):
    # Filtra una lista de eventos comprendida entre dos fechas (date[0] y date[1])
    
    listEventsFiltered = [event for event in listEvents if 
                          isodate.parse(event['tiempo']).replace(tzinfo=None) >= dates[0] 
                          and isodate.parse(event['tiempo']).replace(tzinfo=None) < dates[1]]
    return listEventsFiltered
    
"""

#Funcion que filtra una lista de eventos en funcion de las fechas pasadas por argumento
def filterListByDate(listEvents, dates):
    import numpy
    import pandas as pd

    # Transformamos la lista de eventos a dataframe
    df = pd.DataFrame(listEvents)
    # Convertimos el tiempo a datetime
    df['tiempo'] = pd.to_datetime(df['tiempo'])
    # Comparamos los datos que se encuentren entre las fechas indicadas
    df = df[(df['tiempo'] >= dates[0]) & (df['tiempo'] < dates[1])]
    # Tomamos los indices donde se cumple la condicion
    indices = numpy.array(df.index.values)
    # Tomamos la lista de eventos
    listaF = numpy.array(listEvents)
    # Devolvemos la lista de eventos en dichos indices
    return list(listaF[indices])

def dateFiltering(eventsDF, dates):
    # Filtra un dataframe de eventos generando un nuevo dataframe con los eventos
    #  comprendidos entre dos fechas (date[0] y date[1]) no hay eventos por parte de ese usuario 
    # devuelve lista vacia
    
    eventsDFFiltered = eventsDF.copy()
    eventsDFFiltered['Eventos'] = eventsDF['Eventos'].apply(lambda x: filterListByDate(x, dates))
    
    return eventsDFFiltered

In [ ]:
#devuelve el diccionario de dataframes con los eventos por fechas 
def timeFiltering(eventsDF, dateDict = {}):
    #  Devuelve un diccionario de caracteristicas.
    
    # Inicializacion del diccionario de caracteristicas
    eventsDFSplit = {}
    
    # Si no hay diccionario, cuenta entre todas las fechas (diccionario por defecto)
    if dateDict == {}:
        dateDict = {"ALL": []}
    
    # Recorremos los distintos periodos de tiempo dados (ej: semanas) y extraemos las caracteristicas
    for periods in dateDict:
        eventsDFSplit[periods] = dateFiltering(eventsDF, dateDict[periods])
        
    return eventsDFSplit

In [ ]:
"""
Descartado porque ya no requiere fechas para trabajar
def eventCounter(eventsDF, dates = []):
    # Cuenta eventos en un rango de fechas (date[0] y date[1])
    
    # Filtrado por fecha si dates no es vacio.
    if dates != []:
        eventsDFFiltered = dateFiltering(eventsDF, dates)
    else:
        eventsDFFiltered = eventsDF

    # Contar eventos
    featuresDF = eventsDFFiltered.copy()
    featuresDF['NumEventos'] = featuresDF['Eventos'].apply(lambda x: len(x))
    del featuresDF['Eventos']
    
    return featuresDF
"""

#funcion que cuenta el numero de eventos y lo inclute en una columna de eventsDF
def eventCounter(eventsDF):
    # Contar eventos
    featuresDF = eventsDF.copy()
    featuresDF['NumEventos'] = featuresDF['Eventos'].apply(lambda x: len(x))
    del featuresDF['Eventos']
    
    return featuresDF


"""
Descartado porque ya no va a recibir un dataframe entero. Separacion de filtrado
por tiempo y extraccion de caracteristicas.

def eventCounterMD(eventsDF, dateDict = {}):
    # Multi-Date. Cuenta eventos dado un diccionario de fechas dateDict. 
    #  Devuelve un diccionario de caracteristicas.
    
    # Inicializacion del diccionario de caracteristicas
    featuresDict = {}
    
    # Si no hay diccionario, cuenta entre todas las fechas (diccionario por defecto)
    if dateDict == {}:
        dateDict = {"ALL": []}
    
    # Recorremos los distintos periodos de tiempo dados (ej: semanas) y extraemos las caracteristicas
    for periods in dateDict:
        featuresDict[periods] = eventCounter(eventsDF, dateDict[periods])
        
    return featuresDict
"""

In [ ]:
def hasNotPerformedAnyEvent(eventsDF, dates, kind = []):
    # Devuelve si el estudiante no ha realizado algun evento en un rango de fechas.
    # Si kind es [] valora cualquier tipo de evento, si no, los tipos de eventos pasados
    # en dicha lista.
    
    # Filtramos por rango de fechas
    eventsDFFiltered = dateFiltering(eventsDF, dates)
    
    # Si kind esta lleno, filtramos por la lista de eventos
    if kind != []:
        eventsDFFiltered = eventsFiltering(eventsDFFiltered, kind)

    # Contamos el numero de eventos    
    returnDF = eventCounter(eventsDFFiltered)
    
    # Si no ha hecho eventos, ponemos a True
    returnDF["target"] = (returnDF["NumEventos"] < 1).map(int)
        
    del returnDF['NumEventos']
    
    return returnDF

In [ ]:
#devuleve el target de abandono a 1 si el usuario no ha hecho actividades evaluables en las proximas dos semanas
def dropoutCrit1(eventsDF, dateDict = {}):
    
    # Inicializacion del diccionario de caracteristicas target
    targetDict = {}
    
    # Definimos los eventos evaluables
    evaluable = ['openassessmentblock.self_assess', 'problem_check']

    # Ordenamos el diccionario, en el caso en el que no estuviese ordenado
    import operator
    sortedDict = sorted(dateDict.items(), key=operator.itemgetter(1))
    
    # Recorremos los distintos periodos de tiempo dados (ej: semanas) y extraemos las caracteristicas target
    for ind in range(len(sortedDict)-2): # -2 porque no podemos predecir las dos ultimas semana
        
        # Vamos de la fecha de inicio de la semana siguiente a la fecha del final de dentro de dos semanas
        dates = [sortedDict[ind+1][1][0], sortedDict[ind+2][1][1]]
        
        # Comprobamos si no ha realizado ningun evento evaluable en ese rango de tiempo
        targetDict[sortedDict[ind][0]] = hasNotPerformedAnyEvent(eventsDF, dates, evaluable) 
        
    return targetDict

In [ ]:
#devuelve el target de abandono a 1 si el usuario no ha hecho ningun evento la proxima semana
def dropoutCrit2(eventsDF, dateDict = {}):
    
    # Inicializacion del diccionario de caracteristicas target
    targetDict = {}
    
    # Ordenamos el diccionario, en el caso en el que no estuviese ordenado
    import operator
    sortedDict = sorted(dateDict.items(), key=operator.itemgetter(1))
   
    # Recorremos los distintos periodos de tiempo dados (ej: semanas) y extraemos las caracteristicas target
    for ind in range(len(sortedDict)-1): # -1 porque no podemos predecir la ultima semana
        
        # Vamos de la fecha de inicio y fin de la semana siguiente
        dates = sortedDict[ind+1][1]
        
        
        # Comprobamos si no ha realizado ningun evento en ese rango de tiempo
        targetDict[sortedDict[ind][0]] = hasNotPerformedAnyEvent(eventsDF, dates) 
        
    return targetDict

In [ ]:
#Combina las caracteristicas de la lista de dataframes pasadas por argumento
def combineFeat(featList):
    import pandas as pd
    
    dfold = featList[0].copy()
    
    for ind in range(1,len(featList)):
        dfnew = featList[ind].copy()
        del dfnew['Usuario']
        dfold = pd.concat([dfold, dfnew], axis=1)
    dfold = dfold.dropna(axis = 0,how='any')
    return dfold

In [ ]:
#Diccionario semana
from datetime import datetime
#import dateutil.parser as isodate

### SEMANAS DE LA EDICION 1 ##

sem1 = datetime(2015,2,24)
sem2 = datetime(2015,3,3)
sem3 = datetime(2015,3,10)
sem4 = datetime(2015,3,17)
sem5 = datetime(2015,3,24)
sem6 = datetime(2015,3,31)
sem7 = datetime(2015,4,7)
sem8 = datetime(2015,4,14)
"""

### SEMANAS DE EDICION 2 ##

sem1 = datetime(2015,10,5)
sem2 = datetime(2015,10,12)
sem3 = datetime(2015,10,19)
sem4 = datetime(2015,10,26)
sem5 = datetime(2015,11,2)
sem6 = datetime(2015,11,9)
sem7 = datetime(2015,11,16)
sem8 = datetime(2015,11,23)
"""

SEM = {"Week1": [sem1, sem2],
       "Week2": [sem2, sem3],
       "Week3": [sem3, sem4],
       "Week4": [sem4, sem5],
       "Week5": [sem5, sem6],
       "Week6": [sem6, sem7]}

In [ ]:
# Cargamos el fichero de trabajo
filename = "Edicion1/eventos_final.json"

eventsDFRaw = readDataFile(filename)

#Se eliminan los que no tengan id de usuario
eventsDFRaw = eventsDFRaw[eventsDFRaw['Usuario'] != '']
print("Tamaño:",len(eventsDFRaw))
print(eventsDFRaw.head(10))  

#### Filtramos los usuarios en funcion de un número de eventos

In [ ]:
# Filtrado por usuario: eliminamos los que hayan realizado
#  menos de 50 eventos en el curso
eventsDF = dataFiltering(eventsDFRaw, 50);
print("Tamaño:",len(eventsDF))   
print(eventsDF.head(10))

#### Filtramos en intervalos de tiempo obteniendo un diccionario de dataframes por semanas

In [ ]:
# Filtrado por tiempo
eventsWeek = timeFiltering(eventsDF, SEM)

# Filtrado por usuario
for week in eventsWeek:
    eventsWeek[week] = dataFiltering(eventsWeek[week],0)
    
eventsWeek

### Extracción de características
En este apartado se procede a extraer las características por semana.

**Número de eventos**: Número total de eventos que ha generado un usuario (por semana).

In [ ]:
# Num eventos x semana
numEvents = {}

for week in eventsWeek:
    # Contamos eventos
    numEvents[week] = eventCounter(eventsWeek[week])
numEvents    

**Número de interacciones de vídeo**: Número de veces que un usuario interacciona con vídeos (por semana).

Los eventos que se contemplan son: play_video, pause_video, seek_video, stop_video.

In [ ]:
# Num eventos video x semana
numVideos = {}
listEvents = ["play_video", "pause_video", "seek_video", "stop_video"]

for week in eventsWeek:
    # Filtramos por evento
    numVideos[week] = eventsFiltering(eventsWeek[week], listEvents)
    # Contamos eventos
    numVideos[week] = eventCounter(numVideos[week])
    numVideos[week].rename(columns={'NumEventos': 'NumInteraccVideos'}, inplace=True)

**Número de interacciones de problemas**: Número de veces que un usuario interacciona con cualquier tipo de problema -actividades o ejercicios- (por semana).

Los eventos que se contemplan son: problem_check.

In [ ]:
# Num eventos problemas (ejercicios + actividades) x semana
numProbs= {}
listEvents = ["problem_check"]

for week in eventsWeek:
    # Filtramos por evento
    numProbs[week] = eventsFiltering(eventsWeek[week], listEvents)
    # Contamos eventos
    numProbs[week] = eventCounter(numProbs[week])
    numProbs[week].rename(columns={'NumEventos': 'NumProblemas'}, inplace=True)

**Número de problemas correctos**: Número de veces que un usuario hace correctamente un problema -actividades o ejercicios- (por semana).

Los eventos que se contemplan son: problem_check.

In [ ]:

#Funcion que cuenta el numero de problemas correctos en una lista
def countCorrectByList(listResProblem):
    
    #A veces la lista de resultados puede venir vacía asique lo tomamos como resultado no correcto (esto pasa muy pocas veces debe ser un error del sevidor de edX)
    if listResProblem == []:
        return 0;
    
    dfL = pd.DataFrame(listResProblem)
    
    countCorrecto = dfL[dfL["correcto"]=="True"]["correcto"].count()
    return countCorrecto
    
    
#Cuenta el numero de eventos correctos a partir de una lista de eventos   
def correctProblemByList(listEvents):
    
    #Si tenemos lista de eventos vacia contamos 0 problemas correctos
    if listEvents==[]:
        return 0
    
    #Transformamos la lista de eventos a dataframe
    df = pd.DataFrame(listEvents)

    #tenemos una lista de resultados por cada problema ya que estan formados por subejercicios,
    #por lo tanto tomamos la lista y la transformamos a dataframe en la funcion del apply
    df['numCorrectByUser'] = df['resultados'].apply(lambda x: countCorrectByList(x))
 
    return  df['numCorrectByUser'].sum()

def correctProblems(eventsDF):
    

    
    eventsDFFiltered = eventsDF.copy()
    eventsDFFiltered['numCorrect'] = eventsDF['Eventos'].apply(lambda x: correctProblemByList(x))
    del eventsDFFiltered['Eventos']
    
    return eventsDFFiltered

In [ ]:
import numpy
import pandas as pd

correctProbs= {}
listEvents = ["problem_check"]

for week in eventsWeek:
    # Filtramos por evento
    correctProbs[week] = eventsFiltering(eventsWeek[week], listEvents)
    # Contamos eventos
    correctProbs[week] = correctProblems(correctProbs[week])
    

correctProbs


**Número de intentos en problemas**: Número de veces que un usuario intenta un problema -actividades o ejercicios- (por semana).

Los eventos que se contemplan son: problem_check.

In [ ]:
    
    
def attempsProblemByList(listEvents):
    
        #Si tenemos lista de eventos vacia contamos 0 intentos
        if listEvents==[]:
            return 0

        #Transformamos la lista de eventos a dataframe
        df = pd.DataFrame(listEvents)

        return  pd.to_numeric(df['num_intentos']).sum()

def attempProblems(eventsDF):
    
        import numpy
        import pandas as pd

        eventsDFFiltered = eventsDF.copy()
        eventsDFFiltered['numAttemps'] = eventsDF['Eventos'].apply(lambda x: attempsProblemByList(x))
        del eventsDFFiltered['Eventos']

        return eventsDFFiltered

In [ ]:
attempsProbs= {}
listEvents = ["problem_check"]

for week in eventsWeek:
    # Filtramos por evento
    attempsProbs[week] = eventsFiltering(eventsWeek[week], listEvents)
    # Contamos eventos
    attempsProbs[week] = attempProblems(attempsProbs[week])
    
    

attempsProbs


In [1]:
#ESTO ESTARÍA BIEN PASARLO A UN FICHERO YA QUE SOLO ES PARA EL CURSO DE ANDROID
#CONTIENE LA INFORMACIÓN TEMPORAL DE LOS PROBLEMAS

autoevaluacion = 14, #La autoevaluacion hasta el problema 


# ANGEL: REVISION
dicProblems = {
"Week1" : "47", #la semana 1 hasta el problema 47 incluido...
"Week2" : "79",
"Week3" : "91",
"Week4" : "109",
"Week5" : "133",
"Week6" : "138"}


problemsByWeek=[47,79,91,109,133,138]
problemsByWeek[0]

47

**Numero de problemas correctos autoevaluación de java**

Los eventos que se contemplan son: problem_check.

In [2]:
def correctProblemByListAE(listEvents):
    
    #Si tenemos lista de eventos vacia contamos 0 problemas correctos
    if listEvents==[]:
        return 0
    
    
    #Transformamos la lista de eventos a dataframe
    df = pd.DataFrame(listEvents)
    
    #pasamos el id natural del problema a numerico
    df[['id_natural']] = df[['id_natural']].apply(pd.to_numeric)
    
    #comprobamos que el id_natural del problema este dentro de los de la autoevaluacion
    df = df[df['id_natural'] < autoevaluacion]

    #tenemos una lista de resultados por cada problema ya que estan formados por subejercicios,
    #por lo tanto tomamos la lista y la transformamos a dataframe en la funcion del apply
    df['numCorrectByUser'] = df['resultados'].apply(lambda x: countCorrectByList(x))
 
    return  df['numCorrectByUser'].sum()

def correctProblemsAE(eventsDF):
    

    
    eventsDFFiltered = eventsDF.copy()
    eventsDFFiltered['numCorrectAE'] = eventsDF['Eventos'].apply(lambda x: correctProblemByListAE(x))
    del eventsDFFiltered['Eventos']
    
    return eventsDFFiltered

In [3]:
listEvents = ["problem_check"]

#Aqui filtramos los eventos para cualquier intervalo de tiempo
#,no necesitamos el diccionario de dataframe, ya que la 
#autoevaluacion se puede hacer tambien en otra semana que no sea la primera 
eventsProbAE = eventsFiltering(eventsDF, listEvents)


eventsProbAE = correctProblemsAE(eventsProbAE)


eventsProbAE


NameError: name 'eventsFiltering' is not defined

**Nota proyecto**: Nota del proyecto de los usuarios por semana

Los eventos que se contemplan son: openassessmentblock.self_assess.

In [ ]:
#Funcion que suma los puntos de las partes de la autoevaluacion
def sumPoints(listPartsPoints):
    
    #A veces la lista de resultados puede venir vacía asique lo tomamos como 0 puntos
    if listPartsPoints == []:
        return 0.0;
    
    dfL = pd.DataFrame(listPartsPoints)
    dfL[['puntos_autoevaluacion']] = dfL[['puntos_autoevaluacion']].apply(pd.to_numeric)
     
    return dfL["puntos_autoevaluacion"].sum()  
    
    
    
def sumPointsByList(listEvents):
    
    #Si tenemos lista de eventos vacia contamos que tiene un 0.0 en el proyecto
    if listEvents==[]:
        return 0.0
    
    #Transformamos la lista de eventos a dataframe
    df = pd.DataFrame(listEvents)

   
    df['self-evaluationPoints'] = df['partes'].apply(lambda x: sumPoints(x))
 
    return  df['self-evaluationPoints'].sum()

def pointsSE(eventsDF):
    
    import pandas as pd

    
    eventsDFFiltered = eventsDF.copy()
    eventsDFFiltered['self-evaluationPoints'] = eventsDF['Eventos'].apply(lambda x: sumPointsByList(x))
    del eventsDFFiltered['Eventos']
    
    return eventsDFFiltered

In [ ]:

pointsProyect = {}

#lista de eventos por los que filtramos
listEvents = ["openassessmentblock.self_assess"]

for week in eventsWeek:
    # Filtramos por evento
    pointsProyect[week] = eventsFiltering(eventsWeek[week], listEvents)
    # Contamos eventos
    pointsProyect[week] = pointsSE(pointsProyect[week])
    
    
#pointsProyect    

In [ ]:

def countWordsByList(listEvents):
    
    #Si tenemos lista de eventos vacia contamos que tiene 0 palabras
    if listEvents==[]:
        return 0.0
    
    #Transformamos la lista de eventos a dataframe
    df = pd.DataFrame(listEvents)

    #Contamos el numero de palabras y lo guardamos en una nueva columna llamada Total_words
    df['Total_words'] = df['cuerpo'].apply(lambda x: len(x.split()))
 
    return  df['Total_words'].sum()

def wordsForumCount(eventsDF):
    
    
    eventsDFFiltered = eventsDF.copy()
    eventsDFFiltered['Total words'] = eventsDF['Eventos'].apply(lambda x: countWordsByList(x))
    del eventsDFFiltered['Eventos']
    
    return eventsDFFiltered

In [ ]:

import pandas as pd

wordsForum = {}
listEvents = ["edx.forum.comment.created", "edx.forum.response.created", "edx.forum.thread.created"]

for week in eventsWeek:
    # Filtramos por evento
    wordsForum[week] = eventsFiltering(eventsWeek[week], listEvents)
    # Contamos eventos
    wordsForum[week] = wordsForumCount(wordsForum[week])
    
    
#wordsForum



**Número de interacciones con proyecto**: Número de veces que un usuario hace un submit de una actividad de autoevaluacion -proyecto- (por semana).

Los eventos que se contemplan son: openassessmentblock.self_assess.

In [ ]:
# Num eventos proyecto x semana
numProyecto = {}
listEvents = ["openassessmentblock.self_assess"]

for week in eventsWeek:
    # Filtramos por evento
    numProyecto[week] = eventsFiltering(eventsWeek[week], listEvents)
    # Contamos eventos
    numProyecto[week] = eventCounter(numProyecto[week])
    numProyecto[week].rename(columns={'NumEventos': 'NumProyecto'}, inplace=True)

**Número de interacciones con documentos**: Número de veces que un usuario interacciona con un documento (por semana).

Los eventos que se contemplan son: textbook.pdf.chapter.navigated.

In [ ]:
# Num eventos documentos x semana
numDocs = {}
listEvents = ["textbook.pdf.chapter.navigated"]

for week in eventsWeek:
    # Filtramos por evento
    numDocs[week] = eventsFiltering(eventsWeek[week], listEvents)
    # Contamos eventos
    numDocs[week] = eventCounter(numDocs[week])
    numDocs[week].rename(columns={'NumEventos': 'NumDocs'}, inplace=True)

**Número de interacciones con foro**: Número de veces que un usuario interacciona con el foro del curso (por semana).

Los eventos que se contemplan son: edx.forum.searched,edx.forum.comment.created, edx.forum.response.created, edx.forum.thread.created.

In [ ]:
# Num eventos foro x semana
numForo = {}
listEvents = ["edx.forum.searched","edx.forum.comment.created", "edx.forum.response.created", "edx.forum.thread.created"]

for week in eventsWeek:
    # Filtramos por evento
    numForo[week] = eventsFiltering(eventsWeek[week], listEvents)
    # Contamos eventos
    numForo[week] = eventCounter(numForo[week])
    numForo[week].rename(columns={'NumEventos': 'numInteraccForo'}, inplace=True)


#### A contuanción en la siguiente celda concatenamos los dataframes con los eventos por semana

In [ ]:
featMatrix = {}

for week in eventsWeek:
    featMatrix[week] = combineFeat([numEvents[week], numVideos[week], numProbs[week], 
                                    numProyecto[week], numDocs[week], numForo[week],
                                    wordsForum[week],pointsProyect[week],attempsProbs[week],correctProbs[week],
                                    ])
    
featMatrix

**AQUI VAN LAS CARACTAERISTICAS DE LOS PROBLEMAS DEL TFG DE MIGUEL**

**En primer lugar hay que recorrer el dataframe inicial con todos los problemas para ver cada problema cuantos ejercicios tiene, lo incluiremos en un diccionario.** 

**ATENCIÓN ESTO YA NO HACE FALTA HACERLO YA QUE LO CARGAMOS DE UN FICHERO CON EL ID DEL PROBLEMA Y SU NUMERO DE EJERCICIOS!!!!!
**

In [ ]:
import pandas as pd
def countNumberEj(row,dictNumEj):
    
    if int(row['id_natural']) in dictNumEj:
        return dictNumEj;
    
    else:
        dictNumEj[int(row['id_natural'])] = int(row['num_ejercicios'])
        return dictNumEj
        
    return dictNumEj

def countNumberEjByList(listEvents,dictNumEj):
    
        #Si tenemos lista de eventos vacia devolvemos el diccionario tal y como venia
        if listEvents==[]:
            return

        #Transformamos la lista de eventos a dataframe
        df = pd.DataFrame(listEvents)[["id_natural","num_ejercicios"]]
        #print(df)
        
        df.apply(lambda x: countNumberEj(x,dictNumEj), axis=1)
        
        return
    
    
def createDictByList(eventsDF,dictNumEj):
    
    eventsDF['Eventos'].apply(lambda x: countNumberEjByList(x,dictNumEj))
 
    return  dictNumEj

In [ ]:
"""
listEvents = ["problem_check"]
dictNumEj = {}

problemsAllWeek = eventsFiltering(eventsDF, listEvents)


dictNumEj = createDictByList(problemsAllWeek,dictNumEj)
print(dictNumEj)

"""

#Ya no hace falta hacerlo lo leemos de un fichero.

#### Cargamos de un fichero el id del problema y el numero de ejercicios en un diccionario

In [ ]:
import json

#with open('dictNumEj.json', 'w') as fp: #Para guardar el dict de numero
       #json.dump(dictNumEj, fp)
    
#tenemos que tener en cuenta que en la segunda edicion del curso de android tenemos menos problemas tenemos que pasarle el diccionario de la edicion1
#con los numEjercicios.

with open('dictNumEj.json') as data_file:
     dictNumEj = json.load(data_file)
        
dictNumEj = {int(k):int(v) for k,v in dictNumEj.items()}
dictNumEj

**Funciones auxiliares que necesitamos para las características de los problemas**

In [ ]:
#Transforma un diccionario pasado por argumento en un dataframe de una sola fila para unirlo a otro
def dictToDataframe(dictToDf):
   
    
    return pd.DataFrame(dictToDf,index=[0,])

#Crea un diccionario con numero de intentos a 0 y resultado del problema a -1
def createDictByNameProblems(dfProblems):
    
    dictNamePro = {}
    listColumnName = dfProblems.columns
    
    for name in listColumnName:
        
        if name.find("Intentos"):
            dictNamePro[name] = -1
        else:
            dictNamePro[name] = 0 
    return dictNamePro

#Crea un dataframe vacio pero con el nombre de las columnas entre el problema inicial y el final
def createDataframeProblems(problemIni,problemFin):
    listProblems = []

    for numPr in range(problemIni-1,int(problemFin)):

        for numEj in range(dictNumEj[numPr+1]):
            stringIntentos = "Intentos_"+str(numPr+1)+"_"+str(numEj+1)
            stringPr = "Problema_"+str(numPr+1)+"_"+str(numEj+1)
            listProblems.append(stringIntentos)
            listProblems.append(stringPr)

    listProblems.insert(0,"Usuario")

    dfProblems = pd.DataFrame(columns=listProblems)
    return dfProblems

**Funciones de las características de los problemas**

In [ ]:
def countCorrectProblemsByList(listResProblem,dictNamePro):
    
   
    #A veces la lista resultado puede venir vacia por error de edX por lo tanto salimos de la función
    if listResProblem["resultados"] == []:
        return 
    
    #AQUI HAY QUE RECORRER LOS RESULTADOS MODIFICANDO EL DICCIONARIO
    intento = 1
    for resultado in listResProblem["resultados"]:
        dictNamePro["Intentos_"+listResProblem["id_natural"]+"_"+str(intento)]= listResProblem["num_intentos"]
        
        if resultado["correcto"] == "True":
            dictNamePro["Problema_"+listResProblem["id_natural"]+"_"+str(intento)]= 1
        else:
            dictNamePro["Problema_"+listResProblem["id_natural"]+"_"+str(intento)]= 0

        intento = intento+1
        
    
    return dictNamePro
    
    
    
def problemsFeaturesByList(row,dictName):
    
    global dfPro_Empty
    
   
    user =  row["Usuario"]
    
    dictNamePro = dictName.copy()
    dictNamePro["Usuario"]=user
    
    #Si tenemos lista de eventos vacia no ha hecho problemas asique incluimos 0 y -1 altenativamente
    if row["Eventos"]==[]:
        rowDf = dictToDataframe(dictNamePro)
        dfPro_Empty = dfPro_Empty.append(rowDf)
        return
         
    
    df = pd.DataFrame(row["Eventos"])
    df.apply(lambda x: countCorrectProblemsByList(x,dictNamePro),axis=1)
    
    rowDf = dictToDataframe(dictNamePro)
    dfPro_Empty = dfPro_Empty.append(rowDf)
    
    return

def problemsFeatures(eventsDF,dictName):
    
    
    eventsDFFiltered = eventsDF.copy()
    
    eventsDF.apply(lambda row: problemsFeaturesByList(row,dictName),axis=1)
    
    
    return dfPro_Empty

#### ATENCIÓN LA SIGUIENTE CELDA ES PARA TENER EN CUENTA SOLO LOS PROBLEMAS DE LA SEMANA ACTUAL, ES DECIR CUENTA LOS PROBLEMAS CORRECTO, INCORRECTOS O NO REALIZADOS SOLO SI LOS PROBLEMAS SON DE ESA SEMANA.!!! NO ESTAN UTILIZADAS ESTAS CARACTERÍSTICAS EN EL TFM DE MIGUEL

In [ ]:
"""
listEvents = ["problem_check"]

df_Problems= {}
df_Pr={} #Final
problemsByWeek=[0,47,79,91,109,133,138] #numero del ultimo problema de esa semana
indexList = 0

for week in sorted(eventsWeek):
   
    #Filtrar eventos por semana
    df_Problems[week] = eventsFiltering(eventsWeek[week], listEvents)
    
    #Crear dataframe vacio con lista de atributos de los problemas, num_intentos,y resultado del problema
    dfPro_Empty = createDataframeProblems(problemsByWeek[indexList]+1,problemsByWeek[indexList+1])
    
    #indice para recorrer la lista anterior con el numero del ultimo problema por semana
    indexList = indexList+1
    
    #Nos guardamos el orden de las columnas para ordenar el dataframe despues
    colsOrder = dfPro_Empty.columns
    
    #Crear diccionario con nombre de las columnas, 0 en numero de intentos y -1 en si es correcto para cada problema
    dictName = createDictByNameProblems(dfPro_Empty)
    
    #Extraemos las caracteristicas de los problemas que se guardan en dfPro_Empty como variable global
    problemsFeatures(df_Problems[week],dictName)
    
    #Resetamos el indice y ordenamos las columnas
    dfPro_Empty = dfPro_Empty.reset_index(drop=True)
    dfPro_Empty = dfPro_Empty[colsOrder]
    
    #Copiamos el dfPro_Empty al dataframe Final de los problemas
    df_Pr[week]=dfPro_Empty.copy()
   
    
df_Pr
"""

#### ESTAS SI SON LAS CARACTERÍSTICAS DEL TFM DE MIGUEL YA QUE SE TIENEN EN CUENTA TODOS LOS PROBLEMAS INDEPENDIENTEMEN DE LA SEMANA A LA QUE PERTENECE EL PROBLEMA

In [ ]:
##ESTO ES PARA LOS PROBLEMAS NO ACUMULADOS ES DECIR LOS QUE HA HECHO ESA SEMANA INDEPENDIENTEMENTE DE LA SEMANA A LA QUE
## PERTENEZCA EL PROBELMA



listEvents = ["problem_check"]

df_Problems= {} #Angel: AUX
df_PrNoAc={} #Final
problemsByWeek=[0,47,79,91,109,133,138] #numero del ultimo problema de esa semana
indexList = 0

for week in sorted(eventsWeek):
   
    #Filtrar eventos por semana
    df_Problems[week] = eventsFiltering(eventsWeek[week], listEvents)
    
    #Crear dataframe vacio con lista de atributos de los problemas, num_intentos,y resultado del problema
    dfPro_Empty = createDataframeProblems(1,problemsByWeek[indexList+1])
    
    #indice para recorrer la lista anterior con el numero del ultimo problema por semana
    indexList = indexList+1
    
    #Nos guardamos el orden de las columnas para ordenar el dataframe despues
    colsOrder = dfPro_Empty.columns
    
    #Crear diccionario con nombre de las columnas, 0 en numero de intentos y -1 en si es correcto para cada problema
    dictName = createDictByNameProblems(dfPro_Empty)
    
    #Extraemos las caracteristicas de los problemas que se guardan en dfPro_Empty como variable global
    problemsFeatures(df_Problems[week],dictName)
    
    #Resetamos el indice y ordenamos las columnas
    dfPro_Empty = dfPro_Empty.reset_index(drop=True)
    dfPro_Empty = dfPro_Empty[colsOrder]
    
    #Copiamos el dfPro_Empty al dataframe Final de los problemas
    df_PrNoAc[week]=dfPro_Empty.copy()
    df_PrNoAc[week] = df_PrNoAc[week].set_index('Usuario')

   
    
df_PrNoAc

**EN LA SIGUIENTE CELDA RENOMBRAMOS LAS CARACTERISTICAS POR SEMANA**

In [ ]:
for week in eventsWeek:
    featMatrix[week].rename(columns={'NumEventos': 'NumEventos_'+week,
                                     'NumInteraccVideos': 'NumInteraccVideos_'+week,
                                     'NumProblemas': 'NumProblemas_'+week,
                                     'NumProyecto': 'NumProyecto_'+week,
                                     'NumDocs': 'NumDocs_'+week,
                                     'numInteraccForo': 'numInteraccForo_'+week,
                                     'Total words': 'Total_words_'+week,
                                     'self-evaluationPoints': 'self-evaluationPoints_'+week,
                                     'numAttemps': 'numAttemps_'+week,
                                     'numCorrect': 'numCorrect_'+week}, inplace=True) 
    
featMatrix

** En la siguiente celda seleccionamos el criterio de abandono a tener en cuenta, es decir, el target.**

In [ ]:
# Este criterio de dropout toma el dataframe eventsDF
target = dropoutCrit2(eventsDF, SEM)

for week in sorted(target):
        target[week]=target[week].set_index('Usuario')
target

** EN LA SIGUIENTE CELDA CARGAMOS LA LISTA DE USUARIOS CON SU NOTA EN EL EXAMEN FINAL **

In [ ]:
import pandas as pd
dfNota = csvToDf("NotaUsuariosEd2.csv")
dfNota = dfNota.set_index('Usuario')
dfNota.index = dfNota.index.map(str)
dfNota

**VAMOS A CREAR LOS DATAFRAMES DE PREDICCION POR SEMANAS AÑADIENDO EL TARGET Y LAS CARACTERISTICAS QUE SON LAS MISMAS PARA TODAS LAS SEMANAS POR EJEMPLO EL NUMERO DE PROBLEMAS CORRECTOS EN LA AUTOEVALUACION**

In [ ]:
def combineFeat2(featList): #Este combineFeat combina los dataframe de semanas y el target eliminand aquellos que su usuario es vacio 
    #y añade 0 aquellos que vamos a predecir pero no salen en alguna semana
    import pandas as pd
    
    dfold = featList[0].copy()
    
    for ind in range(1,len(featList)):
        dfnew = featList[ind].copy()
        del dfold['Usuario']
        dfold = pd.concat([dfold, dfnew], axis=1)
    dfold = dfold.dropna(subset=["Usuario"])
    dfold = dfold.fillna(0)
    return dfold

In [ ]:
#FUNCION QUE RECIBE UN DICCIONARIO DE DATAFRAMES Y TRANSFORMA TODAS SUS COLUMNAS A TIPO ENTERO EXCEPTO SI ES LA DE LA NOTA "Final"
def typeIntDf(df):
#Por ultimo convertimos todas las columnas a tipo entero
    for week in df:

        for col in list(df[week].columns):
            if col == "Final":#Estamos prediciendo la nota final asique no tiene que ser un entero
                continue
            df[week][col] = df[week][col].astype(int)
    return df

**La siguiente celda combina las caracteristicas generales a partir del diccionario de dataframes **

In [ ]:
def combinarFeatures(featMatrix):
    
    dfToPredictFeat = {}
    listaSemanas = []
    
    for week in sorted(featMatrix):
        listaSemanas.append(featMatrix[week])

        dfToPredictFeat[week] = combineFeat2(listaSemanas)#Este combine feat añade 0 en caso de que no haya hecho eventos esa semana
            
    return dfToPredictFeat



In [ ]:
dfWithFeat= combinarFeatures(featMatrix)
dfWithFeat

**En la siguiente celda incluimos la nota de la autoevaluacion en java ya que es la misma para todas las semanas**

In [ ]:
def combinarAE(eventsProbAE,feats):
    dfWithFeatAe = {}
    for week in sorted(feats):
        listaSemanas = []
        listaSemanas.append(feats[week])
        listaSemanas.insert(0,eventsProbAE)
        dfWithFeatAe[week] = combineFeat(listaSemanas)
        dfWithFeatAe[week] = dfWithFeatAe[week].set_index('Usuario')
        
    return dfWithFeatAe 




In [ ]:
dfWithFeat = combinarAE(eventsProbAE,dfWithFeat)
dfWithFeat

#### EN LA SIGUIENTE CELDA SE CREA LA FUNCION QUE COMBINA DOS DICCIONARIOS DE CARACTERISTICAS POR SEMANA.
#### ATENCIÓN: EL ÍNDICE DEL DATAFRAME TIENE QUE SER EL ID DEL USUARIO

In [ ]:
#PARA COMBINAR DOS TIPOS CARACTERISTICAS, EJ: Problemas y Transiciones 
#Para ello el indice tienen que ser los usuarios
def combineTwoDict(dict1,dict2):
    
    dfToPredictFeat = {}
    
    for week in sorted(dict1):
                                  
        dfToPredictFeat[week] = pd.concat([dict1[week], dict2[week]], axis=1)
            
    return dfToPredictFeat

In [ ]:
dfToPredictCombined = combineTwoDict(df_PrNoAc,dfWithFeat)
dfToPredictCombined

** EN LA SIGUIENTE CELDA SE CREA UNA FUNCIÓN QUE COMBINA EL TARGET DE LA NOTA CON LOS DATAFRAMES POR SEMANA**

In [ ]:
#COMBINARR la nota SOLO
def combinarNota(target,feats):
    dfWithTarget = {}
    
    for week in sorted(feats):
        
        dfWithTarget[week] = pd.concat([feats[week],target], axis=1)
        dfWithTarget[week] = dfWithTarget[week].dropna(how='any') 
        
    return dfWithTarget   


In [ ]:
dfToPredictPrNoAc = combinarNota(dfNota,df_PrNoAc)
dfToPredictPrNoAc

In [ ]:
dfToPredictFt = combinarNota(dfNota,dfWithFeat)
dfToPredictFt

In [ ]:
dfToPredict2 = combinarNota(dfNota,dfToPredictCombined)
dfToPredict2

** EN LA SIGUIENTE CELDA SE CREA UNA FUNCIÓN QUE COMBINA EL TARGET DEL ABANDONO CON LOS DATAFRAMES POR SEMANA**

In [ ]:
#COMBINARR EL TARGET SOLO
def combinarTarget(target,feats):
    dfWithTarget = {}
    
    for week in sorted(target):
        
        dfWithTarget[week] = pd.concat([target[week],feats[week]], axis=1)
        dfWithTarget[week] = dfWithTarget[week].dropna(how='any') 
    return dfWithTarget   


In [ ]:
#for week in df_Pr:
#    df_Pr[week] = df_Pr[week].set_index('Usuario')
    
#dfToPredictPr = combinarTarget(target,df_Pr)#Problemas sin acumular
#dfToPredictPr

In [ ]:
dfToPredictPrNoAc = combinarTarget(target,df_PrNoAc)
dfToPredictPrNoAc

In [ ]:
dfToPredictFt = combinarTarget(target,dfWithFeat)
dfToPredictFt

In [ ]:
dfToPredict2 = combinarTarget(target,dfToPredictCombined)
dfToPredict2

**RESETEAMOS EL INDICE Y PONEMOS LA COLUMNA USUARIO DE NUEVO
A CONTINACIÓN SE VA A GUARDAR LAS CARACTERÏSTICAS EN FUNCIÓN DEL ARGUMENTO QUE PASEMOS A LA FUCIÓN changeIndexColUser,
POR EJEMPLO SI QUEREMOS GUARDAR LAS CARACTERISTICAS DE LOS PROBLEMAS SOLO TENEMOS QUE PASARLE changeIndexColUser(dfToPredictPrNoAc) **

In [ ]:

def changeIndexColUser(dfDict):
    for week in dfDict:
        dfDict[week].reset_index(level=0, inplace=True)
        dfDict[week].rename(columns={'index': 'Usuario'}, inplace=True) 

    return dfDict

In [ ]:
dfToPredict = changeIndexColUser(dfToPredictFt)#cambiar en funcion del dataframe que guardemos
dfToPredict

**AHORA VAMOS A COLOCAR EL TARGET EN LA ULTIMA POSICIÓN y el usuario en primera**

In [ ]:


def orderColToPredict(dfToPredict,nameTarget):
    dfFinal={}
    for week in dfToPredict:

        columnas = dfToPredict[week].columns.tolist()

        columnas.remove(nameTarget)
        columnas.append(nameTarget)
        columnas.remove("Usuario")
        columnas.insert(0,"Usuario")

        dfFinal[week]=dfToPredict[week][columnas]
    return dfFinal
    
 

In [ ]:
dfFinal = orderColToPredict(dfToPredict,"Final")
dfFinal["Week1"]

In [ ]:
#TRANSFORMAMOS TODAS LAS COLUMNAS A ENTERO MENOS EN CASO DE LA NOTA
dfFinal = typeIntDf(dfFinal)
dfFinal

**Ahora guardamos los diccionarios en csv para predecir**

In [ ]:
def dicToCsv(name,dictW):
    for week in dictW:
        filename= name+"_"+week+".csv"
        dfToCsv(dictW[week],filename)
 

In [ ]:
dicToCsv("NuevosAtributos",dfFinal)

In [ ]:
dfleido = csvToDf("semana_Week1.csv");
dfleido